# Introduction
In this research project, we focusing on how to classify the DNA sequence by using deep learning models and benefit from deep learning capability of extracting significant features. Deep learning uses these features to improve the classification accuracy. Deep learning has advanced rapidly and have state-of-the-art performance in bioinformatics field. In genomic analysis, DNA sequence classification is one of the most important tasks. DNA sequence with bases [A, C, G, T] carry a class label that identifies the kind of the sequences. DNA sequence classification has many challenges the main challenge is feature selection. Even with new feature selection techniques, feature selection is a major problem in sequence classification. Using deep learning techniques, there is no need for manual feature extraction, deep learning extract features automatically and classify them. So, we need to decide which architecture will be able to produce a good results in DNA classification task by implement a model to deal with this problem and select how the data will be represented to fed in deep learning network.

# Necessary library/package imports

In [1]:
import pandas as pd
import numpy as np
from keras.layers import Dense, Conv1D, MaxPool1D, LSTM, Conv2D, MaxPool2D, ConvLSTM2D, ConvRNN2D, TimeDistributed, Flatten
from keras.activations import relu, sigmoid
from keras.models import Sequential
from tensorflow.contrib import learn
from keras.utils import to_categorical
from keras.utils import plot_model
import matplotlib.pyplot as plt
import glob

Using TensorFlow backend.


# Data reading and preprocessing

In [2]:
data = np.genfromtxt(fname='histone/H3.txt', dtype='str')

The data is read in the following format:

data[0] = Some kind of sequence ID

data[1] = DNA Sequence of A, T, C, and G

data[2] = Binary Label (0 or 1)

We only require the sequence, and the labels, therefore, we'll ommit the indexes 0, 3, 6, 9, ... 3*n.

In [3]:
# Slicing the data to get rid of indexes with unwanted information.
unwanted = data[0::3]
data = [x for x in data if x not in unwanted]

Let's see if we're now left only with the sequences, and the labels, or not.

In [4]:
data[0:10]

['CACACCACACCCACACACCCACACACCACACCACACACCACACCACACCCACACACACACATCCTAACACTACCCTAACACAGCCCTAATCTAACCCTGGCCAACCTGTCTCTCAACTTACCCTCCATTACCCTGCCTCCACTCGTTACCCTGTCCCATTCAACCATACCACTCCGAACCACCATCCATCCCTCTACTTACTACCACTCACCCACCGTTACCCTCCAATTACCCATATCCAACCCACTGCCACTTACCCTACCATTACCCTACCATCCACCATGACCTACTCACCATACTGTTCTTCTAC',
 '0',
 'AATTTTTATAGGTCGACCCTTCTGTCGCTTACTGGGTTGATTATCTTGTGCTTTCTTAGTATCTATCACAAAGGAGACAAAATCGTTGATAAAAAGTGCATCAACATTCCCAGCCAGAAAATGCACATCATAAAGACATGTTATTCAAGAGCCACGACCGTCTTCAATTTATCTTTTATAAAAAACCCTTGTTCTACTGACAGGATGGAATAGATATTAAATATACATTTTGCATTTTTTTTTTTTTCTGTATTGAAGATTTGTATATGAAAGATGTTTATACATCAAATGCTTTGAATAAAGCCATCTTAATTTCAATTTCATGCCCTCCTTCACCGTTTTCTGTTGGTCTAGAGGTAGCTTGTTGTGGTCACTAATGAGAACTTAAATAGTTTTCAACTGCTGGTGGTAAATCAATAATTTATGTTCTTAACCTAACATTTGATGACCTTTGATGCGTTGGTTATGTTGAAGACAAATTGCCTCTAATCAGTTCCATT',
 '0',
 'AATTATATTTCCATCAGCTCAATACCGCAGTACTTTGAAACCTGATTTATATATTGCAGAACTTAATTAAAAGTACATTGTAGTTCAAAAAATAAATATCAAACTTTTGGACCCTCTCTTATTGCCTCCCAATTAATTAAAACATCTTTTCTTCCAATCTACAGGT

Great! Now, the next thing would be, that we split this into sequences, and labels. Also, for the output, we'll need to append a 0 if the output is already 1, and vice versa, so that our target variable looks like [0, 1], [1, 0], etc.

In [5]:
sequences = data[0::2]
outputs = data[1::2]

In [6]:
sequences[0:5]

['CACACCACACCCACACACCCACACACCACACCACACACCACACCACACCCACACACACACATCCTAACACTACCCTAACACAGCCCTAATCTAACCCTGGCCAACCTGTCTCTCAACTTACCCTCCATTACCCTGCCTCCACTCGTTACCCTGTCCCATTCAACCATACCACTCCGAACCACCATCCATCCCTCTACTTACTACCACTCACCCACCGTTACCCTCCAATTACCCATATCCAACCCACTGCCACTTACCCTACCATTACCCTACCATCCACCATGACCTACTCACCATACTGTTCTTCTAC',
 'AATTTTTATAGGTCGACCCTTCTGTCGCTTACTGGGTTGATTATCTTGTGCTTTCTTAGTATCTATCACAAAGGAGACAAAATCGTTGATAAAAAGTGCATCAACATTCCCAGCCAGAAAATGCACATCATAAAGACATGTTATTCAAGAGCCACGACCGTCTTCAATTTATCTTTTATAAAAAACCCTTGTTCTACTGACAGGATGGAATAGATATTAAATATACATTTTGCATTTTTTTTTTTTTCTGTATTGAAGATTTGTATATGAAAGATGTTTATACATCAAATGCTTTGAATAAAGCCATCTTAATTTCAATTTCATGCCCTCCTTCACCGTTTTCTGTTGGTCTAGAGGTAGCTTGTTGTGGTCACTAATGAGAACTTAAATAGTTTTCAACTGCTGGTGGTAAATCAATAATTTATGTTCTTAACCTAACATTTGATGACCTTTGATGCGTTGGTTATGTTGAAGACAAATTGCCTCTAATCAGTTCCATT',
 'AATTATATTTCCATCAGCTCAATACCGCAGTACTTTGAAACCTGATTTATATATTGCAGAACTTAATTAAAAGTACATTGTAGTTCAAAAAATAAATATCAAACTTTTGGACCCTCTCTTATTGCCTCCCAATTAATTAAAACATCTTTTCTTCCAATCTACAGGTTTGAAAAGGTAA

In [7]:
outputs[0:5]

['0', '0', '0', '0', '1']

In [8]:
# function to append either 0, or 1 at the end of an existing 1 or 0 label
def append_output(label):
    if label == '0':
        return [0, 1]
    else:
        return [1, 0]

In [9]:
labels = list(map(append_output, outputs))

In [10]:
labels[0:5]

[[0, 1], [0, 1], [0, 1], [0, 1], [1, 0]]

By now, we pretty much have the data converted in the form, from where we can get started with one-hot-encoding of the sequences.
For that purpose, we'll be needing the maximum length of the sequences.

In [11]:
maxSeqLength = max(list(map(len, sequences)))

In [12]:
maxSeqLength

500

In [13]:
#Converting all the sequences like "ATCCGT" into lists, like [A, T, C, C, G, T]
sequences = list(map(list, [[x for x in y] for y in sequences]))

In [14]:
vocab_processor = learn.preprocessing.VocabularyProcessor(maxSeqLength)

In [15]:
# Vocabulary processor maps every word (character, in our case) to an ID.
seq_proc = np.array(list(vocab_processor.fit_transform(list(map(" ".join, sequences)))))

In [16]:
seq_proc[0]

array([1, 2, 1, 2, 1, 1, 2, 1, 2, 1, 1, 1, 2, 1, 2, 1, 2, 1, 1, 1, 2, 1, 2,
       1, 2, 1, 1, 2, 1, 2, 1, 1, 2, 1, 2, 1, 2, 1, 1, 2, 1, 2, 1, 1, 2, 1,
       2, 1, 1, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 3, 1, 1, 3, 2, 2, 1, 2,
       1, 3, 2, 1, 1, 1, 3, 2, 2, 1, 2, 1, 2, 4, 1, 1, 1, 3, 2, 2, 3, 1, 3,
       2, 2, 1, 1, 1, 3, 4, 4, 1, 1, 2, 2, 1, 1, 3, 4, 3, 1, 3, 1, 3, 1, 2,
       2, 1, 3, 3, 2, 1, 1, 1, 3, 1, 1, 2, 3, 3, 2, 1, 1, 1, 3, 4, 1, 1, 3,
       1, 1, 2, 1, 3, 1, 4, 3, 3, 2, 1, 1, 1, 3, 4, 3, 1, 1, 1, 2, 3, 3, 1,
       2, 2, 1, 1, 2, 3, 2, 1, 1, 2, 1, 3, 1, 1, 4, 2, 2, 1, 1, 2, 1, 1, 2,
       3, 1, 1, 2, 3, 1, 1, 1, 3, 1, 3, 2, 1, 3, 3, 2, 1, 3, 2, 1, 1, 2, 1,
       3, 1, 2, 1, 1, 1, 2, 1, 1, 4, 3, 3, 2, 1, 1, 1, 3, 1, 1, 2, 2, 3, 3,
       2, 1, 1, 1, 2, 3, 2, 3, 1, 1, 2, 2, 1, 1, 1, 2, 1, 3, 4, 1, 1, 2, 1,
       3, 3, 2, 1, 1, 1, 3, 2, 1, 1, 2, 3, 3, 2, 1, 1, 1, 3, 2, 1, 1, 2, 3,
       1, 1, 2, 1, 1, 2, 3, 4, 2, 1, 1, 3, 2, 1, 3, 1, 2, 1, 1, 2, 3, 2, 1,
       3, 4,

The above few lines mean, that the characters are mapped into IDs in the following way:

C -> 1

A -> 2

T -> 3

G -> 4

As for the zeroes, the length of the first sequence is smaller than 500, therefore, zeroes are padded. In case any sequence of length greater than 500 (which is impossible in our case because we've adjusted the vocabProcessor according to maxLength), then that sequence gets trimmed to get adjusted into a 500-dimensional vector.

In [17]:
# Vocabulary size
vocab_size = len(vocab_processor.vocabulary_)
print(vocab_size)

5


Indicates the vocabulary set {NULL, C, A, T, G}

In [18]:
seq_proc

array([[1, 2, 1, ..., 0, 0, 0],
       [2, 2, 3, ..., 2, 3, 3],
       [2, 2, 3, ..., 2, 3, 3],
       ..., 
       [2, 3, 3, ..., 2, 4, 2],
       [3, 2, 3, ..., 3, 2, 3],
       [3, 3, 4, ..., 4, 2, 2]])

Now's the time to integrate all the above steps into a single function, which takes the file path as input, and returns the preprocessed sequence array, and one output label array.

In [19]:
def preprocess_squence(filename):
    """
    Inputs: A directory name
    Outputs: A numpy array of one hot encoded features, and a numpy array of appended labels, maxSeqLength, and vocab_size
    """
    #print("Files found in the directory: {}".format(dirList))
    data = np.genfromtxt('histone/'+filename, dtype='str')
    # Slicing the data to get rid of indexes with unwanted information.
    #print(data.shape)
    unwanted = data[0::3]
    data = [x for x in data if x not in unwanted]
    sequences = data[0::2]
    outputs = data[1::2]
    
    # function to append either 0, or 1 at the end of an existing 1 or 0 label
    def append_output(label):
        if label == '0':
            return [0, 1]
        else:
            return [1, 0]
        
    labels = list(map(append_output, outputs))
    
    maxSeqLength = max(list(map(len, sequences)))
    #Converting all the sequences like "ATCCGT" into lists, like [A, T, C, C, G, T]
    sequences = list(map(list, [[x for x in y] for y in sequences]))
    
    vocab_processor = learn.preprocessing.VocabularyProcessor(maxSeqLength)
    
    # Vocabulary processor maps every word (character, in our case) to an ID.
    seq_proc = np.array(list(vocab_processor.fit_transform(list(map(" ".join, sequences)))))
    #applying oneHotEncoding
    oneHotFeatures = list(map(to_categorical, seq_proc))
    
    return np.array(oneHotFeatures), np.array(labels)

In [20]:
#Valid file names are:
import os
filelist = os.listdir('histone')
filelist = ['H4.txt']

In [34]:
X, y = preprocess_squence(filename='H3.txt')

Squentially reading rest of the files, and stacking them up to get one single big dataset.

In [35]:
# for f in filelist:
#     print("reading file {}".format(f))
#     Xtemp, ytemp = preprocess_squence(filename=f)
#     X = np.vstack((X, Xtemp))
#     y = np.vstack((y, ytemp))

In [36]:
X

array([[[ 0.,  1.,  0.,  0.,  0.],
        [ 0.,  0.,  1.,  0.,  0.],
        [ 0.,  1.,  0.,  0.,  0.],
        ..., 
        [ 1.,  0.,  0.,  0.,  0.],
        [ 1.,  0.,  0.,  0.,  0.],
        [ 1.,  0.,  0.,  0.,  0.]],

       [[ 0.,  0.,  1.,  0.,  0.],
        [ 0.,  0.,  1.,  0.,  0.],
        [ 0.,  0.,  0.,  1.,  0.],
        ..., 
        [ 0.,  0.,  1.,  0.,  0.],
        [ 0.,  0.,  0.,  1.,  0.],
        [ 0.,  0.,  0.,  1.,  0.]],

       [[ 0.,  0.,  1.,  0.,  0.],
        [ 0.,  0.,  1.,  0.,  0.],
        [ 0.,  0.,  0.,  1.,  0.],
        ..., 
        [ 0.,  0.,  1.,  0.,  0.],
        [ 0.,  0.,  0.,  1.,  0.],
        [ 0.,  0.,  0.,  1.,  0.]],

       ..., 
       [[ 0.,  0.,  1.,  0.,  0.],
        [ 0.,  0.,  0.,  1.,  0.],
        [ 0.,  0.,  0.,  1.,  0.],
        ..., 
        [ 0.,  0.,  1.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  1.],
        [ 0.,  0.,  1.,  0.,  0.]],

       [[ 0.,  0.,  0.,  1.,  0.],
        [ 0.,  0.,  1.,  0.,  0.],
        [ 0.,

In [37]:
X.shape, y.shape

((14965, 500, 5), (14965, 2))

In [38]:
X = X.reshape((X.shape[0], -1, 1, 1))

In [39]:
X

array([[[[ 0.]],

        [[ 1.]],

        [[ 0.]],

        ..., 
        [[ 0.]],

        [[ 0.]],

        [[ 0.]]],


       [[[ 0.]],

        [[ 0.]],

        [[ 1.]],

        ..., 
        [[ 0.]],

        [[ 1.]],

        [[ 0.]]],


       [[[ 0.]],

        [[ 0.]],

        [[ 1.]],

        ..., 
        [[ 0.]],

        [[ 1.]],

        [[ 0.]]],


       ..., 
       [[[ 0.]],

        [[ 0.]],

        [[ 1.]],

        ..., 
        [[ 1.]],

        [[ 0.]],

        [[ 0.]]],


       [[[ 0.]],

        [[ 0.]],

        [[ 0.]],

        ..., 
        [[ 0.]],

        [[ 1.]],

        [[ 0.]]],


       [[[ 0.]],

        [[ 0.]],

        [[ 0.]],

        ..., 
        [[ 1.]],

        [[ 0.]],

        [[ 0.]]]])

In [40]:
X.shape

(14965, 2500, 1, 1)

In [41]:
data_dim = vocab_size

# Model Explanation
In this project, we propose deep learning model with highly connected blocks contains of two convolutional layers and two max pooling layers as hidden layers. Each convolutional layer is followed by one max pooling layer. Convolutional layers are the main core of CNN. It takes the sequence input as 2-dimensional matrix after applying one-hot vector. Then detect the lower level of features until reach high level features. The input layer is  matrix, where L the length of DNA sequence. The initial number of neurons and weights value are randomly selected.
The 2-D convolutions between the kernel vectors wl, of size 2n+1, and the input signal x calculated by the convolutional layers.

The output of convolutional layer is rectified linear units (ReLU).

In max pooling layer the input is partitions to small regions, then the maximum value picked as output. This layer reduce the complexity of following layers. Max pooling takes filter with size and a stride 2x2 as we noticed in the literature that size is normally used.

RNNs such as perceptron and LSTM are a connection among building blocks that designed to exploit sequential information of input data. The main characteristics of RNN layer that capability of handling sequence information over time.

The main goal of activation function is introduce nonlinearity that not exit in convolutional layers. Finally, the predicted output obtained through the softmax layer. Here, we chose ReLU as activation function that applies The function: 
f(x) = max(0, x)

In [42]:
# Going to try out these with different configurations
from keras.optimizers import rmsprop, adam, SGD, Adadelta

In [43]:
# expected input data shape: (batch_size, maxSeqLength, data_dim)
model = Sequential()
model.add(Conv2D(64, input_shape=(2500, 1, 1), activation='relu', kernel_size=1, strides=1))
model.add(MaxPool2D(pool_size=1, strides=1))
model.add(Conv2D(32, activation='relu', kernel_size=1, strides=1))
model.add(MaxPool2D(pool_size=1, strides=1))
model.add(TimeDistributed(LSTM(16, dropout=0.5, activation='relu', stateful=False)))
model.add(Flatten())
#model.add(LSTM(8, dropout=0.5, activation='relu', stateful=False))
#model.add(LSTM(4, dropout=0.5, activation='relu'))
model.add(Dense(2, activation='softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer=rmsprop(lr=0.001),
              metrics=['accuracy'])

In [44]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 2500, 1, 64)       128       
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 2500, 1, 64)       0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 2500, 1, 32)       2080      
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 2500, 1, 32)       0         
_________________________________________________________________
time_distributed_2 (TimeDist (None, 2500, 16)          3136      
_________________________________________________________________
flatten_2 (Flatten)          (None, 40000)             0         
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 80002     
Total para

## Iterative file reading and training

In [45]:
# #####
# # Temporary Stuff
# #model = Sequential()
# model.load_weights(filepath='cnn_H3K79me3.txt_weights.h5')
# #####
print("X, y lengths: {}, {}".format(len(X), len(y)))
split = int(len(X)*0.9)
print("Split: {}".format(split))
print("Shapes X, y: {} {}".format(X.shape, y.shape))
X_train = X[:split]
y_train = y[:split]

print("Lengths train X, y: {} {}".format(len(X_train), len(y_train)))

X_test = X[split:]
y_test = y[split:]
del X, y # to save some space
print("Lengths test X, y: {} {}".format(len(X_test), len(y_test)))

X, y lengths: 14965, 14965
Split: 13468
Shapes X, y: (14965, 2500, 1, 1) (14965, 2)
Lengths train X, y: 13468 13468
Lengths test X, y: 1497 1497


In [48]:
from keras.callbacks import ModelCheckpoint
# checkpoint
filepath="weights-improvement-{epoch:02d}-{val_acc:.2f}.hdf5"
#model.load_weights('weights-improvement-07-0.67.hdf5')
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]
# Fit the model
history = model.fit(X_train, y_train,
      batch_size=64, epochs=20,
      validation_data=(X_test, y_test), callbacks=callbacks_list)
#saving the weights of the model
model.save_weights(filepath='cnn_rnn_dna_weights.h5')

Train on 13468 samples, validate on 1497 samples
Epoch 1/20
13468/13468 [==============================] - 137s 10ms/step - loss: 0.5554 - acc: 0.7349 - val_loss: 0.5959 - val_acc: 0.6546

Epoch 00001: val_acc improved from -inf to 0.65464, saving model to weights-improvement-01-0.65.hdf5
Epoch 2/20
13468/13468 [==============================] - 138s 10ms/step - loss: 0.5193 - acc: 0.7618 - val_loss: 0.5904 - val_acc: 0.6754

Epoch 00002: val_acc improved from 0.65464 to 0.67535, saving model to weights-improvement-02-0.68.hdf5
Epoch 3/20
13468/13468 [==============================] - 138s 10ms/step - loss: 0.5072 - acc: 0.7717 - val_loss: 0.5919 - val_acc: 0.6847

Epoch 00003: val_acc improved from 0.67535 to 0.68470, saving model to weights-improvement-03-0.68.hdf5
Epoch 4/20
13468/13468 [==============================] - 138s 10ms/step - loss: 0.5007 - acc: 0.7787 - val_loss: 0.5936 - val_acc: 0.7281

Epoch 00004: val_acc improved from 0.68470 to 0.72812, saving model to weights-imp

In [52]:
# summarize history for accuracy
plt.figure(figsize=(18, 12))
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [53]:
# summarize history for loss
plt.figure(figsize=(18, 12))
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()